In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from random import shuffle
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
SEED=2
import os

train=pd.read_csv("data.csv")


x = train['id_code']
y = train['diagnosis']


# Any results you write to the current directory are saved as output.

In [2]:
df_X,X_test,df_y,y_test=train_test_split(x, y, test_size=0.2)

X_train,X_valid,y_train,y_valid=train_test_split(df_X, df_y, test_size=0.2)


In [3]:
def load_ben_color(path, sigmaX=20):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [4]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [5]:
import matplotlib.pyplot as plt 
import cv2
train_images=[]
IMG_SIZE=224
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_train):
    
    path=f"dataset/{row}.png"
    image =load_ben_color(path)
    train_images.append(image)

        

Wall time: 0 ns


In [6]:

test_images=[]
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_test):
    
    path=f"dataset/{row}.png"
    image =load_ben_color(path)
    test_images.append(image)

Wall time: 0 ns


<Figure size 1800x1152 with 0 Axes>

In [7]:

valid_images=[]

%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_valid):
    
    path=f"dataset/{row}.png"
    image =load_ben_color(path)
    valid_images.append(image)

Wall time: 0 ns


<Figure size 1800x1152 with 0 Axes>

In [26]:
y_train_dummies=pd.get_dummies(y_train)
y_test_dummies=pd.get_dummies(y_test)
y_valid_dummies=pd.get_dummies(y_valid)
y_train_dummies

,0,1,2,3,4
2853,1,0,0,0,0
914,0,0,1,0,0
707,1,0,0,0,0
1708,1,0,0,0,0
1245,0,1,0,0,0
...,...,...,...,...,...
95,0,1,0,0,0
1025,0,0,0,0,1
1573,1,0,0,0,0
202,0,0,1,0,0


In [8]:
import numpy as np
import time
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.applications.vgg16 import decode_predictions

from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import os

Using TensorFlow backend.


In [18]:
num_Classes=5

train=np.array(train_images)
test=np.array(test_images)
valid=np.array(valid_images)

num_of_samples=train.shape[0]

In [19]:
image_input=Input(shape=(224,224,3))
model=VGG16(input_tensor=image_input, weights='imagenet')

last_layer=model.get_layer('fc2').output
out=Dense(num_Classes,activation='softmax', name='output')(last_layer)

custom_vgg_model=Model(image_input,out)
custom_vgg_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [27]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable=False

In [28]:
custom_vgg_model.compile(loss="categorical_crossentropy", optimizer='Adam',  metrics=['accuracy'])

In [ ]:
t=time.time()

custom_vgg_model.fit(train,y_train_dummies, batch_size=32, epochs=10, verbose=1 ,validation_data=(valid,y_valid_dummies))

print("training time: %s" %(t-time.time()))

Train on 2343 samples, validate on 586 samples
Epoch 1/10
2112/2343 [==========================>...] - ETA: 1:26 - loss: 1.0278 - accuracy: 0.6780

In [ ]:
(loss, accuracy)=custom_vgg_model.evaluate(test,y_test_dummies,verbose=1 ,batch_size=10)


In [24]:

print(loss)
print(accuracy*100)